In [65]:
import time
startTime = time.time()

L = 45
S = 35
K = 55
E = 50
G = 25
A = 30
V = 40

fragmentTolerance = 1 # 해상도1


spectrum=open("C:/Users/user/bio/proteomics/실습자료/XCorr구현실습/spectrum.mgf",'r')

for line in spectrum:
    line = line.rstrip("\n")
    
    if line.startswith("PEPMASS"):
        pepmass = float(line.split("=")[1])
        bin_size = int(pepmass/fragmentTolerance) + 1
        experimental = [0] * bin_size
    elif line[0].isdigit():
        mz = float(line.split(" ")[0])
        index = int(mz/fragmentTolerance)
        
        intensity=int(line.split(" ")[1])
        experimental[index] = intensity

print(experimental)
spectrum.close()


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 0, 0, 0, 0, 7, 0, 0, 0, 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 95, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 81, 0, 0, 0, 0, 29, 0, 0, 0, 0, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 77, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 0, 0, 0, 0, 9, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [55]:
database=open("C:/Users/ADMIN/Desktop/실습자료/XCorr구현실습/database.fasta",'r')

bestPeptide=""
peptides=[]
bestXCorr = 0

for line in database:
    line = line.rstrip("\n")
    if not (line.startswith(">")):
        peptides.append(line)
        
print(peptides)
database.close()

['LSKEGAV', 'VGGGGEVGG', 'LSGAVEK', 'ESEELE', 'LLESGGK']


In [3]:
print(peptides)
print(len(peptides))
print(peptides[0])
print(peptides[0][:1])
print(peptides[0][:2])
print(peptides[0][:3])
print(peptides[0][:4])
print(peptides[0][:5])
print(peptides[0][:6])

['LSKEGAV', 'VGGGGEVGG', 'LSGAVEK', 'ESEELE', 'LLESGGK']
5
LSKEGAV
L
LS
LSK
LSKE
LSKEG
LSKEGA


In [60]:
bestXCorr = 0
startTime = time.time()
for peptide in range(len(peptides)):
    theoretical =[0]*bin_size
    b_ion = 0
    y_ion = 0
    
    for q in range(len(peptides[peptide])-1):
        if peptides[peptide][q] =="L":
            b_ion += L
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="S":
            b_ion += S
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="K":
            b_ion += K
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="E":
            b_ion += E
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="G":
            b_ion += G
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="A":
            b_ion += A
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="V":
            b_ion += V
            y_ion = pepmass - b_ion
        indexb = int(b_ion/fragmentTolerance)
        indexy = int(y_ion/fragmentTolerance)
        theoretical[indexb] = 1
        theoretical[indexy] = 1
    #print(theoretical)

    tau =74
    xcorr = 0
    xcorr_avg = 0
    
    for t in range(-tau,tau+1):
        for b in range(bin_size):
            if((b+t)>=0 and ((b+t) < bin_size)):
                #print(theoretical[b])
                xcorr_avg += float(experimental[b+t])*float(theoretical[b])
    
    for b in range(bin_size):
        xcorr += experimental[b]*theoretical[b]
    
    xcorr_avg -= xcorr
    xcorr_avg /= 2*tau
    xcorr -= xcorr_avg
    
    if xcorr > bestXCorr:
        bestPeptide = peptides[peptide]
        bestXCorr = xcorr
        
    print(xcorr)
    print("")
    
print(bestPeptide)
print(bestXCorr)
print("running time :",time.time() - startTime)


368.0

55.0

576.0

200.0

454.0

LSGAVEK
576.0
running time : 109.94437885284424


In [13]:
for peptide in range(len(peptides)):
    theoretical =[0]*bin_size
    y_ion = 0
    
    for s in range(len(peptides[peptide])-1):
        reversed_character = len(peptides[peptide])-1-s
        if peptides[peptide][reversed_character] =="L":
            y_ion += L
        if peptides[peptide][reversed_character] =="S":
            y_ion += S
        if peptides[peptide][reversed_character] =="K":
            y_ion += K
        if peptides[peptide][reversed_character] =="E":
            y_ion += E
        if peptides[peptide][reversed_character] =="G":
            y_ion += G
        if peptides[peptide][reversed_character] =="A":
            y_ion += A
        if peptides[peptide][reversed_character] =="V":
            y_ion += V        

        index = int(y_ion/fragmentTolerance)
        theoretical[index] = 1
    print(theoretical)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [64]:

bestXCorr=0
startTime = time.time()

for peptide in range(len(peptides)):
    theoretical =[0]*bin_size
    b_ion = 0
    y_ion = 0
    
    for q in range(len(peptides[peptide])-1):
        if peptides[peptide][q] =="L":
            b_ion += L
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="S":
            b_ion += S
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="K":
            b_ion += K
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="E":
            b_ion += E
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="G":
            b_ion += G
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="A":
            b_ion += A
            y_ion = pepmass - b_ion
        if peptides[peptide][q] =="V":
            b_ion += V
            y_ion = pepmass - b_ion
        indexb = int(b_ion/fragmentTolerance)
        indexy = int(y_ion/fragmentTolerance)
        theoretical[indexb] = 1
        theoretical[indexy] = 1
    #print(theoretical)
    
    tau = 74
    intensity_sum = 0
    xcorr_sum = 0
    xcorr_avg = 0
    xcorr = 0
    
    # tau + n

    for c in range(tau):
        intensity_sum += experimental[c]
    for c in range(tau, bin_size+tau):
        if c < bin_size:
            intensity_sum += experimental[c]
        if c >= (2*tau)+1:
            intensity_sum -= experimental[c-(2*tau+1)]
        xcorr_sum += theoretical[c-tau]*intensity_sum
        
    for c in range(bin_size):
        xcorr += experimental[c] * theoretical[c]
        
    xcorr_avg = (xcorr_sum - xcorr) / (2*tau)
    xcorr -= xcorr_avg

    
    if xcorr > bestXCorr:
        bestPeptide = peptides[peptide]
        bestXCorr = xcorr
        
    print(xcorr)
    print("")
    
print(bestPeptide)
print(bestXCorr)
print("running time :",time.time() - startTime)

368.0

55.0

576.0

200.0

454.0

LSGAVEK
576.0
running time : 0.09981846809387207


In [ ]:
'''
tn
fragmentTolerance = 1 일때 0.0979s
fragmentTolerance = 0.1 일때 1.0921s
fragmentTolerance = 0.01 일때 11.1047s
fragmentTolerance = 0.001 일때 109.944s

t+n
fragmentTolerance = 1 일때 0.001s
fragmentTolerance = 0.1 일때 0.01s
fragmentTolerance = 0.01 일때 0.1s

'''
